In [263]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as mpl
import numpy as np
import scipy.stats as st
import calendar
from sklearn.mixture import GaussianMixture

new_cases_df = pd.read_csv("Assignment 2_datasets\Assignment 2_datasets/1.COVID-19_daily_number_of_new_cases_and_deaths.csv", sep=",")
vaccination_df = pd.read_csv("Assignment 2_datasets/Assignment 2_datasets/2.COVID-19_vaccination.csv", sep=",")
icu_df = pd.read_csv("Assignment 2_datasets\Assignment 2_datasets/3.COVID-19_hospital_and_ICU_admission_rates.csv", sep=",")

# Cleaning process
- get rid of unessesary data
- drop duplicates
- missing values
    - Checked for NaN rows in each column individually
    - cases and deaths are the only columns with missing values in new_case_df
- check that colunms are consistent/formating
- outliers



In [264]:
# find_nan = new_case_df[new_cases_df.isnull()==True]
new_cases_df[new_cases_df.isna().any(axis=1)]
# print(new_cases_df.groupby('countriesAndTerritories').count())
# print(new_cases_df[new_cases_df['deaths'].isna()].groupby('countriesAndTerritories').count())
# print(new_cases_df[['countriesAndTerriztories', 'popData2020']].count())

# Store the population per country in a new variable
# print(population_per_country.keys())
# belgien = new_cases_df[new_cases_df["countriesAndTerritories"] == "Belgium"]
# belgien[belgien.isna().any(axis=1)]
# dateRep	day	month	year	cases	deaths	countriesAndTerritories	geoId	countryterritoryCode	popData2020	continentExp# 

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
1432,20/07/2021,20,7,2021,1954.0,NaN,Belgium,BE,BEL,11522440,Europe
1433,19/07/2021,19,7,2021,2134.0,NaN,Belgium,BE,BEL,11522440,Europe
1435,17/07/2021,17,7,2021,1104.0,NaN,Belgium,BE,BEL,11522440,Europe
1439,13/07/2021,13,7,2021,1429.0,NaN,Belgium,BE,BEL,11522440,Europe
1441,11/07/2021,11,7,2021,781.0,NaN,Belgium,BE,BEL,11522440,Europe
...,...,...,...,...,...,...,...,...,...,...,...
26915,11/04/2022,11,4,2022,NaN,38.0,Spain,ES,ESP,47332614,Europe
26916,10/04/2022,10,4,2022,NaN,47.0,Spain,ES,ESP,47332614,Europe
26917,09/04/2022,9,4,2022,NaN,42.0,Spain,ES,ESP,47332614,Europe
26919,07/04/2022,7,4,2022,NaN,53.0,Spain,ES,ESP,47332614,Europe


In [265]:
#Removing Liechtenstein because it has 18% Nan values in the deaths column
new_cases_df = new_cases_df[new_cases_df["countriesAndTerritories"] != "Liechtenstein"]
print(new_cases_df.groupby('countriesAndTerritories').count())


                         dateRep   day  month  year  cases  deaths  geoId  \
countriesAndTerritories                                                     
Austria                      971   971    971   971    971     971    971   
Belgium                      968   968    968   968    968     952    968   
Bulgaria                     962   962    962   962    962     954    962   
Croatia                      972   972    972   972    972     972    972   
Cyprus                       794   794    794   794    794     794    794   
Czechia                     1003  1003   1003  1003   1003    1002   1003   
Denmark                      931   931    931   931    913     931    931   
Estonia                      973   973    973   973    972     951    973   
Finland                     1024  1024   1024  1024   1024    1024   1024   
France                      1006  1006   1006  1006   1006    1006   1006   
Germany                      992   992    992   992    992     992    992   

In [266]:
belgien = new_cases_df[new_cases_df["countriesAndTerritories"] == "Belgium"]["deaths"]
frequency = belgien.value_counts()
print(frequency)
max_value_key = frequency.idxmax()
print(max_value_key)

3.0      49
5.0      48
7.0      47
4.0      40
6.0      37
         ..
160.0     1
159.0     1
134.0     1
139.0     1
82.0      1
Name: deaths, Length: 154, dtype: int64
3.0


In [267]:
# removing nan values from deaths column by replacing them with the most frequent value
all_countries = new_cases_df["countriesAndTerritories"].unique()

print("Number of missing values before: ",new_cases_df["deaths"].isna().sum())

for country in all_countries:
    deaths = new_cases_df[new_cases_df["countriesAndTerritories"] == country]["deaths"]
    frequency = deaths.value_counts().idxmax()
    new_cases_df.loc[new_cases_df["countriesAndTerritories"] == country, "deaths"] = new_cases_df.loc[new_cases_df["countriesAndTerritories"] == country, "deaths"].fillna(frequency)

print("Number of missing values after: ",new_cases_df["deaths"].isna().sum())

Number of missing values before:  126
Number of missing values after:  0


### From the block below we can see that there are no outliars in the popData2020 column because there's only one unique value for each country.

In [271]:
import scipy.stats as st

population_per_country = new_cases_df.groupby("countriesAndTerritories")["popData2020"].first()

for country in population_per_country.keys():
    # print(country)
    # print(new_cases_df[new_cases_df["countriesAndTerritories"] == country]["popData2020"].unique())
    pass


deaths_per_country = new_cases_df.groupby("countriesAndTerritories")["deaths"].sum()
deaths_per_country_avg = new_cases_df.groupby("countriesAndTerritories")["deaths"].mean().round(0)

for country in deaths_per_country.keys():
    print(country)
    print(new_cases_df[new_cases_df["countriesAndTerritories"] == country]["deaths"])
    pass

# # Group the filtered data by month
# filtered_df["dateRep"] = pd.to_datetime(filtered_df["dateRep"])  # Convert the "dateRep" column to datetime if necessary
# filtered_df["Month"] = filtered_df["dateRep"].dt.strftime("%B %Y")
# grouped_df = filtered_df.groupby("Month")["deaths"]
# # Extract all values for each group
# grouped_values = grouped_df.apply(lambda x: x.values)

# print(grouped_values[1])
# # Perform Shapiro-Wilk test and print p-values
# for month, data in grouped_values.items():
#     print(f"data", data)
#     statistic, p_value = st.shapiro(data)
#     print(f"Shapiro-Wilk test for {month}:")
#     print(f"P-value: {p_value:.4f}")
#     print()



Austria
0      0.0
1      4.0
2      4.0
3      6.0
4      8.0
      ... 
966    0.0
967    0.0
968    0.0
969    0.0
970    0.0
Name: deaths, Length: 971, dtype: float64
Belgium
971      3.0
972      9.0
973      6.0
974      6.0
975     14.0
        ... 
1934     3.0
1935     3.0
1936     3.0
1937     3.0
1938     3.0
Name: deaths, Length: 968, dtype: float64
Bulgaria
1939     0.0
1940     1.0
1941     0.0
1942     0.0
1943    11.0
        ... 
2896     0.0
2897     0.0
2898     1.0
2899     0.0
2900     0.0
Name: deaths, Length: 962, dtype: float64
Croatia
2901     9.0
2902     6.0
2903    10.0
2904     7.0
2905     7.0
        ... 
3868     0.0
3869     0.0
3870     0.0
3871     0.0
3872     0.0
Name: deaths, Length: 972, dtype: float64
Cyprus
3873    2.0
3874    2.0
3875    5.0
3876    2.0
3877    2.0
       ... 
4662    0.0
4663    0.0
4664    0.0
4665    0.0
4666    0.0
Name: deaths, Length: 794, dtype: float64
Czechia
4667     0.0
4668     1.0
4669    11.0
4670    11.0
4671    